In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [ ]:
import numpy as np

def square_l2_distance(a:np.ndarray, b:np.ndarray):
    diff = a - b
    return diff.dot(diff)

size = 1000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
)

query = np.random.random(size = 3)

cnt = df.select(
    pds.query_similar_count(
        query = query,
        target = "x1",
        metric = "sqzl2",
        threshold = 0.1
    )
).item(0, 0)

x1 = df["x1"].to_numpy()
actual_cnt = 0
for i in range(0, len(x1) - len(query) + 1):
    sl = x1[i:i+len(query)]
    znormed = (sl - np.mean(sl)) / np.std(sl, ddof=1)
    
    actual_cnt += int((square_l2_distance(znormed, query) < 0.1))

# assert cnt == actual_cnt

In [ ]:
size = 5000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
    pds.random(0., 1.).alias("x2"),
    pds.random(0., 1.).alias("x3"),
).with_row_index().with_columns(
    y = pl.col("x1") + pl.col("x2") * 0.2 - 0.3 * pl.col("x3"),
)

In [ ]:
import numpy as np

query = np.random.random(size = 5)

In [ ]:
query

In [ ]:
df.select(
    pds.query_similar_count(
        query = query,
        target = "x1",
        metric = "sql2",
        threshold = 0.5
    )
)

In [ ]:
df.select(
    pds.query_similar_count(
        query = [0.5, 0.5, 0.1, 0.1, 0.12, 0.22],
        target = "x1",
        metric = "sql2",
        threshold = 0.1
    )
)

In [ ]:
q = pl.Series([0.5, 1.0, 0.3])
qq = pl.lit(q)
df.select(
    (qq - qq.mean()) / qq.std()
)